[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ottolau/CloudChamberTrackClassification/blob/master/keras/cloudchamberAI_training_v2_colab.ipynb)

In [0]:
#!kill -9 -1

Google colab libaray imports

In [0]:
using_colab = True

In [0]:
if using_colab:
    from google.colab import drive
    from google.colab import files as cfiles
    from googleapiclient.http import MediaFileUpload
    
    !nvidia-smi

Python libaray imports

In [0]:
"""Python file path, image, and data processing libraries."""
import random
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [0]:
"""Deep learning libraries."""
import tensorflow as tf
import keras
from keras import backend as K
from keras.optimizers import Adam, SGD, Adagrad, Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, CSVLogger
from keras.models import Model, Sequential, load_model, model_from_json
from keras.layers import Flatten, Dense, Activation, Input, Dropout, Activation, BatchNormalization, Reshape
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras import regularizers
from keras.utils.np_utils import to_categorical
from imgaug import augmenters as iaa


In [0]:
"""Sklearn functions that will help training"""
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

Select pretrained model

In [0]:
PRETRAINED_MODEL = "VGG19"

if PRETRAINED_MODEL == "ResNet50":
    # Keras ResNet routines
    from keras.applications.resnet50 import ResNet50  # Import the ResNet deep neural network
    #from keras.preprocessing import image  # Routines for loading image data
    from keras.applications.resnet50 import preprocess_input  # ResNet-specific routines for preprocessing images
    #from keras.applications.resnet50 import decode_predictions  # ResNet-specific routines for extracting predictions

if PRETRAINED_MODEL == "VGG16":
    # Keras VGG16 routines
    from keras.applications.vgg16 import VGG16
    from keras.applications.vgg16 import preprocess_input
    #from keras.applications.vgg16 import decode_predictions
    
if PRETRAINED_MODEL == "VGG19":
    # Keras VGG19 routines
    from keras.applications.vgg19 import VGG19
    from keras.applications.vgg19 import preprocess_input
    #from keras.applications.vgg19 import decode_predictions
    
if PRETRAINED_MODEL == "InceptionV3":
    # Keras InceptionV3 routines
    from keras.applications.inceptionV3 import InceptionV3
    from keras.applications.inceptionV3 import preprocess_input
    #from keras.applications.inceptionV3 import decode_predictions
    
if PRETRAINED_MODEL == "InceptionResNetV2":
    # Keras InceptionResNetV2 routines
    from keras.applications.inception_resnet_v2 import InceptionResNetV2
    from keras.applications.inception_resnet_v2 import preprocess_input
    #from keras.applications.inception_resnet_v2 import decode_predictions

Globals variables and constants

In [0]:
VERSION = 'v0'  # Model version
NUM_CLASSES = 3
INPUT_SHAPE = [256, 256, 3]
#INPUT_SHAPE = [224, 224, 3]
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 120
PRETRAINED = True

DATA_AUGMENTATION = True
DA_INCREMENT = 4
DA_INCREMENT = DA_INCREMENT if DATA_AUGMENTATION else 1

Mount the data. Define file paths, and file inspection

In [0]:
ROOT_DIR = "."
MYDRIVE_DIR = "/content/gdrive/My Drive"
if using_colab:
    NUM_WORKERS = 2
    ROOT_DIR = "/content"
    drive.mount("/content/gdrive", force_remount=True)
    # After executing the cell above, Drive
    # files will be present in "/content/drive/My Drive".
    !ls "/content/gdrive/My Drive"
    input_csv_path = os.path.join(MYDRIVE_DIR, 'Data')
    data_path = MYDRIVE_DIR
    checkpoint_path = MYDRIVE_DIR
    
else:
    input_csv_path = '../Data'
    data_path = '..'
    checkpoint_path = '.'
    
input_csv = 'mapping_all.csv'

labels_path = os.path.join(input_csv_path, input_csv)
#print(os.listdir(data_path))

Load CSV file

In [0]:
train_label_csv = pd.read_csv(labels_path, index_col=False)  # Pandas for reading csv

classWeight = compute_class_weight('balanced', np.unique(train_label_csv.Target), train_label_csv.Target) 
classWeight = dict(enumerate(classWeight))
print(classWeight)

def curate_dataset(data_csv):
    """Convert data csv into a list of dicts."""
    dataset = []
    for name, label in zip(data_csv.Id, data_csv.Target):
        #print(label)
        dataset += [{
            'path': os.path.join(data_path, name),
            'label': label}]
    dataset = np.array(dataset)
    return dataset

train_dataset = curate_dataset(train_label_csv)
print(train_dataset[:10])

Split dataset into training/validation folds for model selection

In [0]:
train_ids, test_ids, train_targets, test_target = train_test_split(
    train_label_csv.Id,
    train_label_csv.Target,
    test_size=0.5,
    random_state=42)
print(len(train_ids))
# print(train_targets)

Create a data generator class for processing and loading data into our model

In [0]:
class DataGenerator:
    """Data generator for feeding data to keras"""
    def __init__(self,
            label_dims=3,
            max_image=255.,
            batch_size=16,
            proc_img_size=[256, 256, 3],
            train=False):
        self.label_dims = label_dims
        self.max_image = max_image
        self.batch_size = batch_size
        self.proc_img_size = proc_img_size  # Crop to this size
        self.train = train

    def build(self, dataset_info, augument=True, pretrained=True):
        """Data processing routines for training."""
        while True:
            random_indexes = np.random.choice(len(dataset_info), self.batch_size)
            batch_images = np.empty(([self.batch_size] + self.proc_img_size))
            batch_labels = np.zeros((self.batch_size, self.label_dims))
            for i, idx in enumerate(random_indexes):
                if pretrained:
                    image = self.load_image(dataset_info[idx]['path'])
                    image = preprocess_input(image)
                    image = self.augmentations(image)
                else:
                    image = self.load_image(dataset_info[idx]['path']).astype(np.float32)
                    image = self.augmentations(image)
                    image /= self.max_image  # Normalize
                    image = np.maximum(np.minimum(image, 1), 0)  # Clip
                batch_images[i] = image
                batch_labels[i][dataset_info[idx]['label']] = 1
            yield batch_images, batch_labels
    
    def load_image(self, path):
        """Preprocess image."""
        if self.proc_img_size[-1] == 3:
            #image = np.array(Image.open(path))[:,:,:3]
            image = Image.open(path)
            image = image.convert('RGB')
            image = np.array(image)[:,:,:3]
            image = cv2.resize(image, (self.proc_img_size[0], self.proc_img_size[1]))
        return image

    def augmentations(self, image):
        """Apply data augmentations to training images."""
        if self.train:
            augment_img = iaa.Sequential([
                iaa.OneOf([
                    iaa.Affine(rotate=0),
                    iaa.Affine(rotate=90),
                    iaa.Affine(rotate=180),
                    iaa.Affine(rotate=270),
                    iaa.Fliplr(0.5),
                    iaa.Flipud(0.5),
                    # iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                    # iaa.PiecewiseAffine(scale=(0.01, 0.05))
                ]),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                #iaa.Multiply((0.5, 1.5), per_channel=0.5),
                iaa.CropToFixedSize(
                    width=self.proc_img_size[0],
                    height=self.proc_img_size[1],
                    position='uniform')],
            random_order=True)
        else:
            augment_img = iaa.Sequential([
                iaa.CropToFixedSize(
                    width=self.proc_img_size[0],
                    height=self.proc_img_size[1],
                    position='center')])
        image_aug = augment_img.augment_image(image)
        return image_aug

    
# Create train/val datagens
train_datagen = DataGenerator(
    batch_size=TRAIN_BATCH_SIZE,
    proc_img_size=INPUT_SHAPE,
    label_dims=NUM_CLASSES,
    train=DATA_AUGMENTATION)
train_datagen = train_datagen.build(
    dataset_info=train_dataset[train_ids.index], pretrained=PRETRAINED)
val_datagen = DataGenerator(
    batch_size=VAL_BATCH_SIZE,
    proc_img_size=INPUT_SHAPE,
    label_dims=NUM_CLASSES,
    train=False)
val_datagen = val_datagen.build(
    dataset_info=train_dataset[test_ids.index], pretrained=PRETRAINED)
# train_datagen = DataGenerator(proc_img_size=[224, 224, 3], imagenet_proc=True)

# Create datagens for inspection
train_inspect_datagen = DataGenerator(
    batch_size=TRAIN_BATCH_SIZE,
    proc_img_size=INPUT_SHAPE,
    label_dims=NUM_CLASSES,
    train=DATA_AUGMENTATION)
train_inspect_datagen = train_inspect_datagen.build(
    dataset_info=train_dataset[train_ids.index], pretrained=False)
val_inspect_datagen = DataGenerator(
    batch_size=VAL_BATCH_SIZE,
    proc_img_size=INPUT_SHAPE,
    label_dims=NUM_CLASSES,
    train=False)
val_inspect_datagen = val_inspect_datagen.build(
    dataset_info=train_dataset[test_ids.index], pretrained=False)

Inspect data

In [0]:
def plot_images(images, labels, title, num_ims=8):
    """Plot mosaic of images with matplotlib."""
    fig, axs = plt.subplots(1, num_ims, figsize=(25,5))
    plt.suptitle(title)
    for idx, (ax, im, lab) in enumerate(zip(axs, images, labels)):
        ax.imshow(im.squeeze())
        ax.axis('off')
        ax.set_title('Label: %s' % np.where(lab)[0])
    # plt.show()  # Only if not executing in ipython notebook
    print('{0} range, min: {1}, max: {2}'.format(title, images.min(), images.max()))

images, labels = next(train_inspect_datagen)
plot_images(images=images, labels=labels, title='Train')
images, labels = next(val_inspect_datagen)
plot_images(images=images, labels=labels, title='Val')

--------------------------------------------------------------------------------------

Build a model

In [0]:
def resnet_layer(
        inputs,
        num_filters=16,
        kernel_size=3,
        strides=1,
        activation='relu',
        batch_normalization=True,
        conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=NUM_CLASSES):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(
        num_classes,
        activation='softmax',
        kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

def build_finetune_model(input_shape, dropout, fc_layers, num_classes, freeze=True):
    """Load pretrained model, add readout layer, fix the convolutional layers."""
    global PRETRAINED_MODEL
    if PRETRAINED_MODEL == "ResNet50":
        base_model = ResNet50(
           weights='imagenet', 
           include_top=False, 
           pooling='avg',
           input_shape=input_shape)
    
    if PRETRAINED_MODEL == "VGG16":
        base_model = VGG16(
           weights='imagenet', 
           include_top=False, 
           pooling='avg',
           input_shape=input_shape)
        
    if PRETRAINED_MODEL == "VGG19":
        base_model = VGG19(
           weights='imagenet', 
           include_top=False, 
           pooling='avg',
           input_shape=input_shape)
        
    if PRETRAINED_MODEL == "InceptionV3":
        base_model = InceptionV3(
           weights='imagenet', 
           include_top=False, 
           pooling='avg',
           input_shape=input_shape)
        
    if PRETRAINED_MODEL == "InceptionResNetV2":
        base_model = InceptionResNetV2(
           weights='imagenet', 
           include_top=False, 
           pooling='avg',
           input_shape=input_shape)
            
    if freeze:
        for layer in base_model.layers:
            layer.trainable = False

    x = base_model.output
    #x = Flatten()(x)
    x = Dropout(0.20)(x)
    
    for fc in fc_layers:
        # Apply batch normalization
        x = BatchNormalization()(x)
        # New FC layer, random init
        #x = Dense(units=fc, input_shape=input_shape, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = Dense(units=fc, input_shape=input_shape, activation='relu', kernel_constraint=max_norm(3))(x)
        #x = Dense(units=fc, input_shape=input_shape, activation='relu')(x)
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
  
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

keras.backend.clear_session()

if PRETRAINED:
    FC_LAYERS  = [128,64,32]  # Add more layers but adding elements to this list
    dropout    = 0.20

    model = build_finetune_model(
        input_shape=INPUT_SHAPE, 
        dropout=dropout, 
        fc_layers=FC_LAYERS, 
        num_classes=NUM_CLASSES)
    print(model.summary())
    
else:
    model = resnet_v2(input_shape=INPUT_SHAPE, depth=56)
    print(model.summary())

Scoring and history functions for training, and model preparation

In [0]:
def show_history(history):
    """Plot training and validation performance."""
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[2].set_title('weighted acc')
    ax[2].plot(history.epoch, history.history["weighted_acc"], label="Weighted train acc")
    ax[2].plot(history.epoch, history.history["val_weighted_acc"], label="Weighted validation acc")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

checkpointer = ModelCheckpoint(
    os.path.join(checkpoint_path, '%s_resnet_model.h5' % VERSION),
    verbose=2,
    save_best_only=True)
earlyStopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    mode='min',
    patience=20,
    verbose=0,
    restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    cooldown=1,
    verbose=1)
model.compile(
    loss='categorical_crossentropy',  
    optimizer=Adam(1e-3),
    metrics=['accuracy'],
    weighted_metrics=['accuracy'])


Train model

In [0]:
num_train_images = len(train_ids)
num_test_images = len(test_ids)
STEP_PER_EPOCH = num_train_images // TRAIN_BATCH_SIZE * DA_INCREMENT
VALIDATION_STEPS = num_test_images // VAL_BATCH_SIZE
#STEP_PER_EPOCH = 10
#VALIDATION_STEPS = 10

if using_colab:
  
    history = model.fit_generator(
        train_datagen,
        #validation_data=next(val_datagen),
        validation_data=val_datagen,
        class_weight=classWeight,
        epochs=EPOCHS, 
        steps_per_epoch=STEP_PER_EPOCH,
        validation_steps=VALIDATION_STEPS,
        #workers=NUM_WORKERS,
        #use_multiprocessing=True,
        verbose=1,
        callbacks=[checkpointer, earlyStopping, reduce_lr])
  
else:
    history = model.fit_generator(
        train_datagen,
        #validation_data=next(val_datagen),
        validation_data=val_datagen,
        class_weight=classWeight,
        epochs=EPOCHS, 
        steps_per_epoch=STEP_PER_EPOCH,
        validation_steps=VALIDATION_STEPS,
        verbose=1,
        callbacks=[checkpointer, earlyStopping, reduce_lr])

Show history

In [0]:
show_history(history)

Confution Matrix and Classification Report

In [0]:
#Y_pred = model.predict_generator(val_datagen, VALIDATION_STEPS)
#y_pred = np.argmax(Y_pred, axis=1)
#print(y_pred)

y_true = []
y_pred = []
steps_done = 0
while steps_done < VALIDATION_STEPS:
    val_datagen_iter = next(val_datagen)
    y_true = y_true + np.argmax(val_datagen_iter[1], axis=1).tolist()
    y_pred = y_pred + np.argmax(model.predict(val_datagen_iter[0]), axis=1).tolist()
    steps_done = steps_done + 1

    
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('Classification Report')
target_names = ['alpha', 'beta', 'muon']
print(classification_report(y_true, y_pred, target_names=target_names))